In [1]:
# Forward-starting swap pricing in a short-rate binomial model
# Model from the prompt:
# - n = 10 periods
# - short rate lattice: r[k,j] = r0 * u**j * d**(k-j)
# - r0 = 5%, u = 1.1, d = 0.9, q = 0.5
# - per-period compounding, payments in arrears
# Swap:
# - Begins at t=1, first payment at t=2, final payment at t=11
# - Receive floating, pay fixed K = 4.5%
# - Notional N = 1,000,000

import numpy as np

def short_rate_lattice(n, r0, u, d):
    """
    Build the recombining short-rate lattice r[k,j] for k=0..n and j=0..k.

    Parameters
    - n: number of periods
    - r0: short rate at (0,0), decimal (e.g., 0.05)
    - u, d: up/down multiplicative factors for the short rate

    Returns
    - list of numpy arrays; rates[k][j] = r[k,j]
    """
    rates = []
    for k in range(n+1):
        row = np.array([r0 * (u**j) * (d**(k-j)) for j in range(k+1)], dtype=float)
        rates.append(row)
    return rates

def zcb_price_from_short_rates(rates, q, T):
    """
    Price a zero-coupon bond maturing at time T via backward induction.

    Parameters
    - rates: list from short_rate_lattice
    - q: risk-neutral up probability (here 0.5)
    - T: maturity (integer, 1..len(rates)-1)

    Returns
    - P(0,T): model discount factor to time T
    """
    # Terminal payoff (unit face)
    P_next = np.ones(T+1, dtype=float)
    # Backward induction: for k = T-1..0
    for k in range(T-1, -1, -1):
        r_k = rates[k]  # length k+1, applies to interval [k, k+1]
        P_curr = np.empty(k+1, dtype=float)
        for j in range(k+1):
            # Child nodes at time k+1: j (down), j+1 (up)
            cont = (1.0 - q) * P_next[j] + q * P_next[j+1]
            P_curr[j] = cont / (1.0 + r_k[j])
        P_next = P_curr
    return float(P_next[0])

def discount_curve(rates, q, T_max):
    """
    Build discount factors P(0,t) for t = 1..T_max.
    """
    return [zcb_price_from_short_rates(rates, q, t) for t in range(1, T_max+1)]

def price_forward_starting_swap(P, start, end, K, N):
    """
    Price a receive-floating, pay-fixed swap with unit accrual per period.

    Parameters
    - P: list or array of discount factors [P(0,1), P(0,2), ..., P(0,T_max)]
    - start: swap start time (integer, e.g., 1)
    - end: swap end time (integer, e.g., 11)
    - K: fixed rate per period (decimal, e.g., 0.045)
    - N: notional

    Returns
    - PV in currency units
    """
    # Floating leg PV (no notional exchange): N * (P(0,start) - P(0,end))
    PV_float = N * (P[start-1] - P[end-1])
    # Fixed leg PV: N * K * sum_{m=start+1..end} P(0,m)
    annuity = sum(P[m-1] for m in range(start+1, end+1))
    PV_fixed = N * K * annuity
    return PV_float - PV_fixed

# ---- Parameters from the prompt ----
n = 10
r0 = 0.05
u  = 1.10
d  = 0.90
q  = 0.50

K = 0.045
N = 1_000_000
start = 1
end   = 11   # first payment at 2, last at 11

# ---- Build rates, discount curve, and price the swap ----
rates = short_rate_lattice(n=end, r0=r0, u=u, d=d)  # build out to end
P = discount_curve(rates, q=q, T_max=end)

pv = price_forward_starting_swap(P, start=start, end=end, K=K, N=N)
print("Discount factors P(0,t) for t=1..11:")
print([round(x, 9) for x in P])
print(f"Swap PV (receive float, pay {K*100:.2f}%): {pv:,.2f}")
print(f"Rounded to nearest integer: {int(round(pv))}")  

Discount factors P(0,t) for t=1..11:
[0.952380952, 0.907050046, 0.86391608, 0.822889574, 0.783882884, 0.746810304, 0.711588151, 0.678134859, 0.646371045, 0.616219581, 0.587605643]
Swap PV (receive float, pay 4.50%): 33,374.24
Rounded to nearest integer: 33374


In [3]:
# Swaption on a short-rate binomial model (correct node-local discounting)
# Model:
#   n = 10, r[k,j] = r0 * u**j * d**(k-j), q = 0.5, per-period compounding
# Swaption:
#   European, expiry = 5; underlying swap (receive float, pay fixed K=4.5%)
#   pays in arrears at t = 6..11, notional N = 1_000_000, strike = 0 (i.e., max(swap value, 0))

import numpy as np

def short_rate_lattice(n, r0, u, d):
    # r[k][j] for k=0..n, j=0..k
    return [np.array([r0 * (u**j) * (d**(k-j)) for j in range(k+1)], dtype=float)
            for k in range(n+1)]

def zcb_from_node(rates, q, k, j, m):
    """
    Discount factor from node (k,j) to maturity m (k <= m).
    Backward induction within the subtree rooted at (k,j).
    """
    if m == k:
        return 1.0
    # terminal at time m: ones at all reachable nodes
    P_next = np.ones(m - k + 1, dtype=float)
    # step back t = m-1 .. k
    for t in range(m-1, k-1, -1):
        s = t - k                      # number of nodes at time t relative to start
        r_t = np.array([rates[t][j+l] for l in range(s+1)], dtype=float)
        P_curr = np.empty(s+1, dtype=float)
        for l in range(s+1):
            cont = (1.0 - q) * P_next[l] + q * P_next[l+1]
            P_curr[l] = cont / (1.0 + r_t[l])
        P_next = P_curr
    return float(P_next[0])

def node_df_vector(rates, q, k, j, m_end):
    """
    All DFs from (k,j) to m=k..m_end inclusive; returns list D with D[m] = P_{(k,j)}(m).
    """
    D = [None] * (m_end + 1)
    for m in range(k, m_end + 1):
        D[m] = zcb_from_node(rates, q, k, j, m)
    return D

def swap_value_at_node(rates, q, k, j, K, N, first_pay, last_pay):
    """
    Value at node (k,j) of receive-floating, pay-fixed swap paying at first_pay..last_pay.
    Unit accrual per period. Uses local DFs from (k,j).
    """
    D = node_df_vector(rates, q, k, j, last_pay)
    # Floating leg PV at time k: N * (1 - P(k,last_pay))
    pv_float = N * (1.0 - D[last_pay])
    # Fixed leg PV: N * K * sum_{m=first_pay..last_pay} P(k,m)
    annuity = sum(D[m] for m in range(first_pay, last_pay + 1))
    pv_fixed = N * K * annuity
    return pv_float - pv_fixed

def price_swaption(n, r0, u, d, q, K, N, expiry, first_pay, last_pay):
    """
    European swaption price at t=0 by backward induction on the option value.
    Payoff at expiry is max(V_swap(expiry,node), 0).
    """
    # Build rates up to the last discounting step (need rates[0..last_pay-1])
    rates = short_rate_lattice(n=last_pay-1, r0=r0, u=u, d=d)

    # Value at expiry across nodes
    V = np.zeros(expiry + 1, dtype=float)
    for j in range(expiry + 1):
        swap_val = swap_value_at_node(rates, q, expiry, j, K, N, first_pay, last_pay)
        V[j] = max(swap_val, 0.0)

    # Backward induction to time 0
    for k in range(expiry - 1, -1, -1):
        V_new = np.empty(k + 1, dtype=float)
        for j in range(k + 1):
            r = rates[k][j]
            cont = (1.0 - q) * V[j] + q * V[j + 1]
            V_new[j] = cont / (1.0 + r)
        V = V_new
    return float(V[0])

# ---- Parameters from the prompt ----
n = 10
r0 = 0.05
u  = 1.10
d  = 0.90
q  = 0.50
K = 0.045
N = 1_000_000
expiry    = 5
first_pay = 6
last_pay  = 11

swaption_pv = price_swaption(n, r0, u, d, q, K, N, expiry, first_pay, last_pay)
print(f"Swaption PV: {swaption_pv:,.2f}")
print(f"Rounded integer: {int(round(swaption_pv))}")

Swaption PV: 26,311.08
Rounded integer: 26311
